In [122]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
# import plotly.express as px
import warnings

warnings.filterwarnings("ignore")

In [123]:
df = pd.read_csv("cardekho_imputated.csv", index_col=[0])

In [124]:
df.head()

,car_name,brand,model,vehicle_age,km_driven,seller_type,fuel_type,transmission_type,mileage,engine,max_power,seats,selling_price
0,Maruti Alto,Maruti,Alto,9,120000,Individual,Petrol,Manual,19.70,796,46.30,5,120000
1,Hyundai Grand,Hyundai,Grand,5,20000,Individual,Petrol,Manual,18.90,1197,82.00,5,550000
2,Hyundai i20,Hyundai,i20,11,60000,Individual,Petrol,Manual,17.00,1197,80.00,5,215000
3,Maruti Alto,Maruti,Alto,9,37000,Individual,Petrol,Manual,20.92,998,67.10,5,226000
4,Ford Ecosport,Ford,Ecosport,6,30000,Dealer,Diesel,Manual,22.77,1498,98.59,5,570000


In [125]:
df.drop(['car_name','brand'],axis=1,inplace=True)

In [126]:
df.isnull().sum()

model                0
vehicle_age          0
km_driven            0
seller_type          0
fuel_type            0
transmission_type    0
mileage              0
engine               0
max_power            0
seats                0
selling_price        0
dtype: int64

In [127]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15411 entries, 0 to 19543
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   model              15411 non-null  object 
 1   vehicle_age        15411 non-null  int64  
 2   km_driven          15411 non-null  int64  
 3   seller_type        15411 non-null  object 
 4   fuel_type          15411 non-null  object 
 5   transmission_type  15411 non-null  object 
 6   mileage            15411 non-null  float64
 7   engine             15411 non-null  int64  
 8   max_power          15411 non-null  float64
 9   seats              15411 non-null  int64  
 10  selling_price      15411 non-null  int64  
dtypes: float64(2), int64(5), object(4)
memory usage: 1.4+ MB


In [128]:
X = df.drop('selling_price',axis=1)
y = df['selling_price']

In [129]:
X

,model,vehicle_age,km_driven,seller_type,fuel_type,transmission_type,mileage,engine,max_power,seats
0,Alto,9,120000,Individual,Petrol,Manual,19.70,796,46.30,5
1,Grand,5,20000,Individual,Petrol,Manual,18.90,1197,82.00,5
2,i20,11,60000,Individual,Petrol,Manual,17.00,1197,80.00,5
3,Alto,9,37000,Individual,Petrol,Manual,20.92,998,67.10,5
4,Ecosport,6,30000,Dealer,Diesel,Manual,22.77,1498,98.59,5
...,...,...,...,...,...,...,...,...,...,...
19537,i10,9,10723,Dealer,Petrol,Manual,19.81,1086,68.05,5
19540,Ertiga,2,18000,Dealer,Petrol,Manual,17.50,1373,91.10,7
19541,Rapid,6,67000,Dealer,Diesel,Manual,21.14,1498,103.52,5
19542,XUV500,5,3800000,Dealer,Diesel,Manual,16.00,2179,140.00,7


In [130]:
y

0         120000
1         550000
2         215000
3         226000
4         570000
          ...   
19537     250000
19540     925000
19541     425000
19542    1225000
19543    1200000
Name: selling_price, Length: 15411, dtype: int64

In [131]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
X['model']=le.fit_transform(X['model'])



In [132]:
num_features = X.select_dtypes(exclude="object").columns
onehot_columns = ['seller_type','fuel_type','transmission_type']

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

numeric_transformer = StandardScaler()
oh_transformer = OneHotEncoder(drop='first')

preprocessor = ColumnTransformer(
    [
        ("OneHotEncoder", oh_transformer, onehot_columns),
        ("StandardScaler", numeric_transformer, num_features)
        
    ],remainder='passthrough'
)

In [133]:
X = preprocessor.fit_transform(X)

In [134]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3)

In [135]:
from sklearn.linear_model import LinearRegression,Lasso,Ridge,ElasticNet
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import r2_score

In [136]:
models = {
    'LinearRegression': LinearRegression(),
    'Lasso':Lasso(),
    "Ridge":Ridge(),
    'ElasticNet':ElasticNet(),
    'RandomForestRegressor':RandomForestRegressor(),
    'DecisionTreeRegressor':DecisionTreeRegressor(),
    'KNeighborsRegressor':KNeighborsRegressor()
}

,fit_intercept,True
,copy_X,True
,tol,1e-06
,n_jobs,None
,positive,False


In [145]:
for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train,y_train)
    y_pred = model.predict(X_test)
    y_pred1 = model.predict(X_train)
    print(f"training accourecy of {list(models.keys())[i]} : ",r2_score(y_train,y_pred1))
    print('-'*35)
    print(f"test accourecy of {list(models.keys())[i]} : ",r2_score(y_test,y_pred))
    print('='*35)

training accourecy of LinearRegression :  0.6221819969801112
-----------------------------------
test accourecy of LinearRegression :  0.6534333985089992
training accourecy of Lasso :  0.6221819899433831
-----------------------------------
test accourecy of Lasso :  0.6534351205666684
training accourecy of Ridge :  0.622180501631009
-----------------------------------
test accourecy of Ridge :  0.6534864430203059
training accourecy of ElasticNet :  0.568397266407008
-----------------------------------
test accourecy of ElasticNet :  0.6359683543804104
training accourecy of RandomForestRegressor :  0.9809436451265618
-----------------------------------
test accourecy of RandomForestRegressor :  0.9307046428800765
training accourecy of DecisionTreeRegressor :  0.9995798730663575
-----------------------------------
test accourecy of DecisionTreeRegressor :  0.8440170032913703
training accourecy of KNeighborsRegressor :  0.8633031103975918
-----------------------------------
test accourecy

In [147]:
rf_model = RandomForestRegressor()
rf_params = {"max_depth": [5, 8, 15, None, 10],
             "max_features": [5, 7, "auto", 8],
             "min_samples_split": [2, 8, 15, 20],
             "n_estimators": [100, 200, 500, 1000]
             }

In [158]:
from sklearn.model_selection import RandomizedSearchCV
try_model = RandomizedSearchCV(estimator = rf_model,param_distributions=rf_params,cv=3,
                                   verbose=2,
                                   n_jobs=-1,
                                   random_state=42)
try_model.fit(X_train,y_train)

Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV] END max_depth=15, max_features=auto, min_samples_split=8, n_estimators=1000; total time=   0.0s
[CV] END max_depth=15, max_features=auto, min_samples_split=8, n_estimators=1000; total time=   0.0s
[CV] END max_depth=15, max_features=auto, min_samples_split=8, n_estimators=1000; total time=   0.0s
[CV] END max_depth=None, max_features=auto, min_samples_split=8, n_estimators=500; total time=   0.0s
[CV] END max_depth=None, max_features=auto, min_samples_split=8, n_estimators=500; total time=   0.0s
[CV] END max_depth=None, max_features=auto, min_samples_split=8, n_estimators=500; total time=   0.0s
[CV] END max_depth=5, max_features=5, min_samples_split=15, n_estimators=200; total time=   1.3s
[CV] END max_depth=5, max_features=5, min_samples_split=15, n_estimators=200; total time=   1.4s
[CV] END max_depth=8, max_features=auto, min_samples_split=20, n_estimators=500; total time=   0.0s
[CV] END max_depth=8, max_features=a

,estimator,RandomForestRegressor()
,param_distributions,"{'max_depth': [5, 8, ...], 'max_features': [5, 7, ...], 'min_samples_split': [2, 8, ...], 'n_estimators': [100, 200, ...]}"
,n_iter,10
,scoring,None
,n_jobs,-1
,refit,True
,cv,3
,verbose,2
,pre_dispatch,'2*n_jobs'
,random_state,42
,error_score,nan


In [155]:
try_model.best_params_

{'n_estimators': 200,
 'min_samples_split': 2,
 'max_features': 5,
 'max_depth': 15}

In [156]:
y_p = try_model.predict(X_test)


In [157]:
r2_score(y_test,y_p)

0.9368173009023558

In [161]:
final_model = RandomForestRegressor(n_estimators=200,min_samples_split=2,max_depth=15,max_features=5)

In [162]:
final_model.fit(X_train,y_train)

,n_estimators,200
,criterion,'squared_error'
,max_depth,15
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,5
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [163]:
y_pred2 = final_model.predict(X_test)

In [164]:
r2_score(y_test,y_pred2)

0.9380866734244068